In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
import joblib

In [ ]:
movies = pd.read_csv("drive/MyDrive/data/movies.csv", encoding="ISO-8859-1")

movies.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731857 entries, 0 to 731856
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    731857 non-null  int64  
 1   title                 731853 non-null  object 
 2   genres                516089 non-null  object 
 3   original_language     731857 non-null  object 
 4   overview              610830 non-null  object 
 5   popularity            731857 non-null  float64
 6   production_companies  340869 non-null  object 
 7   release_date          677487 non-null  object 
 8   budget                731857 non-null  float64
 9   revenue               731857 non-null  float64
 10  runtime               695638 non-null  float64
 11  status                731857 non-null  object 
 12  tagline               108927 non-null  object 
 13  vote_average          731857 non-null  float64
 14  vote_count            731857 non-null  float64
 15  

In [ ]:

movies = movies[(movies['original_language']=='hi') | (movies['original_language']=='en')]
movies[movies.title== 'Jab We Met']
movies.vote_average.value_counts()
movies = movies[movies.vote_average>=4]
movies.popularity.value_counts()
movies = movies[movies.popularity>5]
movies.head(2000)

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,76600,Avatar: The Way of Water,Science Fiction-Adventure-Action,en,Set more than a decade after the events of the...,17043.610,Lightstorm Entertainment-20th Century Studios,2022-12-14,460000000.0,955081686.0,192.0,Released,Return to Pandora.,8.007,1421.0,Sam Worthington-Zoe SaldaÃ±a-Sigourney Weaver-...,loss of loved one-dying and death-alien life-f...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,830784-663712-676701-829799-315162-1037858-881...
1,899112,Violent Night,Action-Comedy-Crime-Thriller,en,When a team of mercenaries breaks into a wealt...,4870.915,Universal Pictures-87North Productions,2022-11-30,20000000.0,55173750.0,112.0,Released,You better watch out.,7.718,535.0,David Harbour-John Leguizamo-Beverly D'Angelo-...,holiday-santa claus-mercenary-saving christmas...,/1XSYOP0JjjyMz1irihvWywro82r.jpg,/g9Kb3RaLjsybI1jpqHQ3QZTCYpB.jpg,837881-668482-661374-860155-877269-791177-7365...
2,411,"The Chronicles of Narnia: The Lion, the Witch ...",Adventure-Family-Fantasy,en,Siblings Lucy Edmund Susan and Peter step thro...,4059.268,Walt Disney Pictures-Walden Media-C.S. Lewis C...,2005-12-07,180000000.0,745013115.0,143.0,Released,Evil Has Reigned For 100 Years...,7.105,9016.0,Skandar Keynes-Georgie Henley-William Moseley-...,saving the world-sibling relationship-witch-ba...,/clfyMMg00OiowO5uupKpVKjRwmV.jpg,/tuDhEdza074bA497bO9WFEPs6O6.jpg,2454-10140-24257-21-1593-953-425-32657-950-809...
3,436270,Black Adam,Action-Fantasy-Science Fiction,en,Nearly 5000 years after he was bestowed with t...,3964.678,New Line Cinema-Flynn Picture Company-Seven Bu...,2022-10-19,200000000.0,389000000.0,125.0,Released,The world needed a hero. It got Black Adam.,7.228,3268.0,Dwayne Johnson-Aldis Hodge-Noah Centineo-Sarah...,lightning-anti hero-superhero-based on comic-d...,/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg,/bQXAqRx2Fgc46uCVWgoPz5L5Dtr.jpg,485470-855440-663712-829799-505642-642721-1023...
5,555604,Guillermo del Toro's Pinocchio,Animation-Fantasy-Drama,en,During the rise of fascism in Mussolini's Ital...,2383.322,The Jim Henson Company-ShadowMachine-Double Da...,2022-11-09,35000000.0,0.0,117.0,Released,Love will give you life.,8.458,1112.0,Gregory Mann-Ewan McGregor-David Bradley-Chris...,italy-based on novel or book-fascism-woodcutte...,/vx1u0uwxdlhV2MUzj4VlcMB0N6m.jpg,/9k7FnCP20c6FsXgBNJpQaB19LiU.jpg,791177-774752-593643-505642-744114-661374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821,9473,"South Park: Bigger, Longer & Uncut",Animation-Comedy,en,When the four boys see an R-rated movie featur...,31.530,Paramount-Warner Bros. Pictures-Scott Rudin Pr...,1999-06-30,21000000.0,83137603.0,81.0,Released,UH-OH.,7.207,2208.0,Trey Parker-Matt Stone-Mary Kay Bergman-Isaac ...,hell-world supremacy-elementary school-atheist...,/tS0PedvA2mFO9VCHYwQpaU1K36U.jpg,/9MS8hAVIEJ1uPpyOmAOcr6fofvm.jpg,39354-12133-784846-997439-400160-3989-8872-496...
2822,11238,Aladdin and the King of Thieves,Adventure-Animation-Family,en,Legendary secrets are revealed as Aladdin and ...,31.526,Disney Television Animation,1996-05-20,0.0,0.0,81.0,Released,The Spectacular Adventure Continues!,6.265,1633.0,Scott Weinger-Linda Larkin-Robin Williams-John...,musical-flying carpet-orient-sequel-genie-arab...,/kTH7qEyvdDYMoQraQZm3LFKCge3.jpg,/ngYzIWyVgDor1TtGZ8fK7AFOdGr.jpg,15969-812-14128-13761-10898-18269-10530-10545-...
2823,913862,Downfall: The Case Against Boeing,Documentary,en,Investigators reveal how Boeingâs alleged pr...,31.523,Moxie Firecracker Films-Imagine Documentaries,2022-02-09,0.0,0.0,89.0,Released,NaN,7.100,108.0,John Fantasia,aviation-airplane crash,/LRPi6b6gpDyKuveut38S8R38NI.jpg,/5CUdP1wrg4S6yZrDHuW3JLohYLO.jpg,946327-923632-928338-20755-853354-653723-32041...
2824,311,Once Upon a Time in America,Drama-Cr

In [ ]:
movies.head(2)
movies = movies[movies['genres']!='Music']
movies.dropna(subset=['genres', 'keywords'], how = 'all', inplace=True)
movies= movies[movies['title'].isna()==False]
movies['genres'].fillna('Unknown_genre', inplace=True)
movies['keywords'].fillna('Unknown_keyword', inplace=True)
movies.reset_index(inplace=True)
movies.drop(['index', 'recommendations'], axis = 1, inplace=True)
# movies.to_csv('movies_cleaned.csv')
movies.isna().sum()
moviesb= movies.copy()
wanted_keywords = movies['keywords'].str.split('-').explode().value_counts().index.tolist()[:500]
wanted_keywords
movies

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path
0,76600,Avatar: The Way of Water,Science Fiction-Adventure-Action,en,Set more than a decade after the events of the...,17043.610,Lightstorm Entertainment-20th Century Studios,2022-12-14,460000000.0,955081686.0,192.0,Released,Return to Pandora.,8.007,1421.0,Sam Worthington-Zoe SaldaÃ±a-Sigourney Weaver-...,loss of loved one-dying and death-alien life-f...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg
1,899112,Violent Night,Action-Comedy-Crime-Thriller,en,When a team of mercenaries breaks into a wealt...,4870.915,Universal Pictures-87North Productions,2022-11-30,20000000.0,55173750.0,112.0,Released,You better watch out.,7.718,535.0,David Harbour-John Leguizamo-Beverly D'Angelo-...,holiday-santa claus-mercenary-saving christmas...,/1XSYOP0JjjyMz1irihvWywro82r.jpg,/g9Kb3RaLjsybI1jpqHQ3QZTCYpB.jpg
2,411,"The Chronicles of Narnia: The Lion, the Witch ...",Adventure-Family-Fantasy,en,Siblings Lucy Edmund Susan and Peter step thro...,4059.268,Walt Disney Pictures-Walden Media-C.S. Lewis C...,2005-12-07,180000000.0,745013115.0,143.0,Released,Evil Has Reigned For 100 Years...,7.105,9016.0,Skandar Keynes-Georgie Henley-William Moseley-...,saving the world-sibling relationship-witch-ba...,/clfyMMg00OiowO5uupKpVKjRwmV.jpg,/tuDhEdza074bA497bO9WFEPs6O6.jpg
3,436270,Black Adam,Action-Fantasy-Science Fiction,en,Nearly 5000 years after he was bestowed with t...,3964.678,New Line Cinema-Flynn Picture Company-Seven Bu...,2022-10-19,200000000.0,389000000.0,125.0,Released,The world needed a hero. It got Black Adam.,7.228,3268.0,Dwayne Johnson-Aldis Hodge-Noah Centineo-Sarah...,lightning-anti hero-superhero-based on comic-d...,/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg,/bQXAqRx2Fgc46uCVWgoPz5L5Dtr.jpg
4,555604,Guillermo del Toro's Pinocchio,Animation-Fantasy-Drama,en,During the rise of fascism in Mussolini's Ital...,2383.322,The Jim Henson Company-ShadowMachine-Double Da...,2022-11-09,35000000.0,0.0,117.0,Released,Love will give you life.,8.458,1112.0,Gregory Mann-Ewan McGregor-David Bradley-Chris...,italy-based on novel or book-fascism-woodcutte...,/vx1u0uwxdlhV2MUzj4VlcMB0N6m.jpg,/9k7FnCP20c6FsXgBNJpQaB19LiU.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22556,435557,The Party's Just Beginning,Drama-Comedy,en,In a bleak Inverness midwinter Luisaidh is car...,5.001,Mt. Hollywood Films-Synchronicity Films,2018-12-07,0.0,0.0,91.0,Released,NaN,6.500,43.0,Karen Gillan-Lee Pace-Matthew Beard-Paul Higgi...,suicide-depression-best friend-drinking-midwin...,/dkQGIm3g4gQIm6sme3P1cHHexnt.jpg,/aDidRYJQE69yUQWFNUtHw6uME09.jpg
22557,35739,Do or Die,Action-Thriller,en,Asian crime boss Kaneshiro captures two volupt...,5.001,Malibu Bay Films,1991-06-28,0.0,0.0,97.0,Released,It's a new kind of game ... The Rules will kil...,4.600,24.0,Pat Morita-Erik Estrada-Dona Speir-Roberta Vas...,assassin-federal agent,/Avtey3NxkTcrnn6youFa6GW0BFm.jpg,/wYZBJFSlAuQ29GmCXqDvTbM6ABZ.jpg
22558,43759,The Boy with Green Hair,Drama-Comedy-Family,en,Peter an orphaned boy is adopted by Gramp Frye...,5.001,RKO Radio Pictures,1948-11-26,0.0,0.0,82.0,Released,Please don't tell why his hair turned green!,6.300,52.0,Pat OâBrien-Robert Ryan-Barbara Hale-Dean St...,hair-pacifist message-conformism-early cold wa...,/a2secYe7o6mDllV6PAgrnaCaCzz.jpg,/axLBB4Q6mgCuy1W0cTfkY4IdUSd.jpg
22559,80775,Away All Boats,War-Drama,en,The story of USS 'Belinda' a U.S. naval ship a...,5.001,Universal Pictures,1956-08-16,0.0,0.0,114.0,Released,The battle cry of the South Pacific,5.400,13.0,Jeff Chandler-George Nader-Lex Barker-Julie Ad...,pacific-naval battle,/dyyRfeB45ZUx9JhFKL0L9FS9wOU.jpg,/6T5WkjSQlakJPq1slvb8WEPV85q.jpg


In [ ]:
moviesb.genres = [[val.strip() for val in sublist] for sublist in moviesb['genres'].str.split('-')]
moviesb.keywords = [[val.strip() for val in sublist] for sublist in moviesb.keywords.str.split('-')]
moviesb[['genres', 'keywords']]
moviesb['keywords'] = moviesb['keywords'].apply(lambda l: [x for x in l if x in wanted_keywords])
moviesb['keywords'].explode().value_counts()

Unknown_keyword           3501
based on novel or book    1505
woman director            1233
murder                    1106
christmas                  717
                          ... 
bank                        46
supernatural power          46
wheelchair                  45
pornography                 45
comedian                    45
Name: keywords, Length: 500, dtype: int64

In [ ]:
moviesb['genres_keywords'] = moviesb['genres']+moviesb['keywords']
moviesb

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,genres_keywords
0,76600,Avatar: The Way of Water,"[Science Fiction, Adventure, Action]",en,Set more than a decade after the events of the...,17043.610,Lightstorm Entertainment-20th Century Studios,2022-12-14,460000000.0,955081686.0,192.0,Released,Return to Pandora.,8.007,1421.0,Sam Worthington-Zoe SaldaÃ±a-Sigourney Weaver-...,"[loss of loved one, dying and death, alien lif...",/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,"[Science Fiction, Adventure, Action, loss of l..."
1,899112,Violent Night,"[Action, Comedy, Crime, Thriller]",en,When a team of mercenaries breaks into a wealt...,4870.915,Universal Pictures-87North Productions,2022-11-30,20000000.0,55173750.0,112.0,Released,You better watch out.,7.718,535.0,David Harbour-John Leguizamo-Beverly D'Angelo-...,"[holiday, santa claus, mercenary, duringcredit...",/1XSYOP0JjjyMz1irihvWywro82r.jpg,/g9Kb3RaLjsybI1jpqHQ3QZTCYpB.jpg,"[Action, Comedy, Crime, Thriller, holiday, san..."
2,411,"The Chronicles of Narnia: The Lion, the Witch ...","[Adventure, Family, Fantasy]",en,Siblings Lucy Edmund Susan and Peter step thro...,4059.268,Walt Disney Pictures-Walden Media-C.S. Lewis C...,2005-12-07,180000000.0,745013115.0,143.0,Released,Evil Has Reigned For 100 Years...,7.105,9016.0,Skandar Keynes-Georgie Henley-William Moseley-...,"[saving the world, sibling relationship, witch...",/clfyMMg00OiowO5uupKpVKjRwmV.jpg,/tuDhEdza074bA497bO9WFEPs6O6.jpg,"[Adventure, Family, Fantasy, saving the world,..."
3,436270,Black Adam,"[Action, Fantasy, Science Fiction]",en,Nearly 5000 years after he was bestowed with t...,3964.678,New Line Cinema-Flynn Picture Company-Seven Bu...,2022-10-19,200000000.0,389000000.0,125.0,Released,The world needed a hero. It got Black Adam.,7.228,3268.0,Dwayne Johnson-Aldis Hodge-Noah Centineo-Sarah...,"[anti hero, superhero, based on comic, demon, ...",/pFlaoHTZeyNkG83vxsAJiGzfSsa.jpg,/bQXAqRx2Fgc46uCVWgoPz5L5Dtr.jpg,"[Action, Fantasy, Science Fiction, anti hero, ..."
4,555604,Guillermo del Toro's Pinocchio,"[Animation, Fantasy, Drama]",en,During the rise of fascism in Mussolini's Ital...,2383.322,The Jim Henson Company-ShadowMachine-Double Da...,2022-11-09,35000000.0,0.0,117.0,Released,Love will give you life.,8.458,1112.0,Gregory Mann-Ewan McGregor-David Bradley-Chris...,"[italy, based on novel or book, musical, stop ...",/vx1u0uwxdlhV2MUzj4VlcMB0N6m.jpg,/9k7FnCP20c6FsXgBNJpQaB19LiU.jpg,"[Animation, Fantasy, Drama, italy, based on no..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22556,435557,The Party's Just Beginning,"[Drama, Comedy]",en,In a bleak Inverness midwinter Luisaidh is car...,5.001,Mt. Hollywood Films-Synchronicity Films,2018-12-07,0.0,0.0,91.0,Released,NaN,6.500,43.0,Karen Gillan-Lee Pace-Matthew Beard-Paul Higgi...,"[suicide, depression, best friend]",/dkQGIm3g4gQIm6sme3P1cHHexnt.jpg,/aDidRYJQE69yUQWFNUtHw6uME09.jpg,"[Drama, Comedy, suicide, depression, best friend]"
22557,35739,Do or Die,"[Action, Thriller]",en,Asian crime boss Kaneshiro captures two volupt...,5.001,Malibu Bay Films,1991-06-28,0.0,0.0,97.0,Released,It's a new kind of game ... The Rules will kil...,4.600,24.0,Pat Morita-Erik Estrada-Dona Speir-Roberta Vas...,[assassin],/Avtey3NxkTcrnn6youFa6GW0BFm.jpg,/wYZBJFSlAuQ29GmCXqDvTbM6ABZ.jpg,"[Action, Thriller, assassin]"
22558,43759,The Boy with Green Hair,"[Drama, Comedy, Family]",en,Peter an orphaned boy is adopted by Gramp Frye...,5.001,RKO Radio Pictures,1948-11-26,0.0,0.0,82.0,Released,Please don't tell why his hair turned green!,6.300,52.0,Pat OâBrien-Robert Ryan-Barbara Hale-Dean St...,[],/a2secYe7o6mDllV6PAgrnaCaCzz.jpg,/axLBB4Q6mgCuy1W0cTfkY4IdUSd.jpg,"[Drama, Comedy, Family]"
22559,80775,Away All Boats,"[War, Drama]",en,The story of USS 'Belinda' a U.S. naval ship a...,5.001,Universal 

In [ ]:

len(moviesb['genres_keywords'].explode().value_counts())
moviesb.original_language.fillna('en')

0        en
1        en
2        en
3        en
4        en
         ..
22556    en
22557    en
22558    en
22559    en
22560    en
Name: original_language, Length: 22561, dtype: object

In [ ]:
# preprocess = Pipeline([('count_vectorizer', CountVectorizer(analyzer=set))])
# pipelines = ColumnTransformer([('vectorizer', preprocess, 'genres_keywords')])
# train_movies = pipelines.fit_transform(moviesb)

train_movies = ColumnTransformer([
                ('cnt_vect', Pipeline([('vectorizer', CountVectorizer(analyzer=set))]), 'genres_keywords'),
                ('cat', Pipeline([('ord', OneHotEncoder()) ]), ['original_language'])]
                ).fit_transform(moviesb)
train_movies
joblib.dump(train_movies, 'train_movies.sav')

['train_movies.sav']

In [ ]:
movies_model = NearestNeighbors(n_neighbors= 100, metric='cosine')
movies_model.fit(train_movies)

NearestNeighbors(metric='cosine', n_neighbors=100)

In [ ]:
sample = train_movies.toarray()[moviesb.title[moviesb['title'].str.lower().str.contains('drishyam')].index[0]]
distances_movies, indices_movies = movies_model.kneighbors([sample])

recommended_movies = moviesb['title'].iloc[indices_movies.squeeze()].tolist()
recommended_movies, distances_movies

(['Drishyam',
  'Ittefaq',
  'Ugly',
  'Dhamaka',
  'The Pledge',
  'Passion',
  'Badla',
  'Te3n',
  'The Big Sleep',
  'Marlowe',
  'Radhe',
  'Deceiver',
  "River's Edge",
  'The Last House on the Left',
  'Don',
  'Woman of Desire',
  'Midnight in the Garden of Good and Evil',
  'A Wednesday!',
  'The Girl on the Train',
  'Chloe',
  'Rustom',
  'Revolver',
  'Blind Date',
  'Bubble',
  'My Son, My Son, What Have Ye Done',
  'Talvar',
  'Raat Akeli Hai',
  'Locked In',
  'Vikram Vedha',
  'Red Corner',
  'In the Electric Mist',
  'Double Indemnity',
  'Murder',
  'Talaash',
  'In the Heat of the Night',
  'Veronica Guerin',
  'Berserk',
  'Soni',
  'Article 15',
  'No One Killed Jessica',
  'A Thursday',
  'Soni',
  'Omkara',
  'Hustle',
  'Mulholland Falls',
  '11:14',
  'I Confess',
  'The Hollow',
  'The Big Clock',
  'Gaslight',
  'The Narrow Margin',
  'And Soon the Darkness',
  "The Cat's Meow",
  'Edge of Darkness',
  'The Heart of Justice',
  'Pride and Glory',
  'Deadfall'

In [ ]:
import json
!pip install memory-profiler
from memory_profiler import memory_usage

def recommend(input='batman'):
    input = [x.strip() for x in (input.lower()).split('|')]


    samples=[]
    for name in input:
        samples.append(train_movies.toarray()[movies.index[movies['title'].str.lower().str.contains(name)]][0])
    sample_mean = [sum(sub_list) / len(sub_list) for sub_list in zip(*samples)]
    distances_movies, indices_movies = movies_model.kneighbors([sample_mean])
    results = movies.loc[indices_movies.squeeze()]
    results['Distances'] = distances_movies.squeeze()
    return json.loads(results.to_json(orient='records'))

train_movies.shape
movies.info()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22561 entries, 0 to 22560
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    22561 non-null  int64  
 1   title                 22561 non-null  object 
 2   genres                22561 non-null  object 
 3   original_language     22561 non-null  object 
 4   overview              22542 non-null  object 
 5   popularity            22561 non-null  float64
 6   production_companies  21187 non-null  object 
 7   release_date          22551 non-null  object 
 8   budget                22561 non-null  float64
 9   revenue               22561 non-null  float64
 10  runtime               22558 non-null  float64
 11  status                22561 non-null  object 
 12  tagline               16164 non-null  object 
 13  vote_average          

In [ ]:
max(memory_usage(proc=recommend))

501.796875

In [ ]:
results_json = recommend('drishyam|andhadhun')
results_df = pd.json_normalize(results_json)
# results_df['English'].tolist()        # ise neeche likhde agar sirf naam ki predictions ki list dekhna chahta h to
results_json
results_df['title'].tolist()

['Drishyam',
 'Dhamaka',
 'Andhadhun',
 'Ittefaq',
 'Ugly',
 'Rustom',
 'Vikram Vedha',
 'An Action Hero',
 'Raat Akeli Hai',
 'Wanted',
 'Darlings',
 'Radhe',
 'Raees',
 'Wazir',
 'Te3n',
 'Baazigar',
 'Hera Pheri',
 'Badla',
 'Article 15',
 'A Thursday',
 'Soni',
 'Soni',
 'No One Killed Jessica',
 'Omkara',
 'Song of the Thin Man',
 'Monica, O My Darling',
 'Special 26',
 'Race',
 'The Pledge',
 'Thar',
 'Raid',
 'Deathtrap',
 'Once Upon a Time in Mumbaai',
 'Haseen Dillruba',
 'Talaash',
 'Holiday',
 'Talvar',
 'Raman Raghav 2.0',
 'Bunty Aur Babli 2',
 'Midnight in the Garden of Good and Evil',
 'The Girl on the Train',
 'Don',
 'Badlapur',
 'In the Electric Mist',
 'Looop Lapeta',
 'Revolver',
 'A Wednesday!',
 'Locked In',
 'Red Corner',
 'Walk All Over Me',
 'Bhool Bhulaiyaa',
 'Blackmail',
 'Blind Date',
 'My Son, My Son, What Have Ye Done',
 'Dhoom',
 "River's Edge",
 'Deceiver',
 'Section 375',
 'Burn Notice: The Fall of Sam Axe',
 'Sooryavanshi',
 'Passion',
 'Jolly LLB 2',

In [ ]:


movies.to_parquet('moviesfile.parquet', compression='GZIP')
joblib.dump(train_movies, 'train_movies.sav')
joblib.dump(movies_model, 'model_movies.sav')


['model_movies.sav']

In [ ]:
# import fastparquet
moviespar = pd.read_parquet('moviesfile.parquet')
moviespar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22561 entries, 0 to 22560
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    22561 non-null  int64  
 1   title                 22561 non-null  object 
 2   genres                22561 non-null  object 
 3   original_language     22561 non-null  object 
 4   overview              22542 non-null  object 
 5   popularity            22561 non-null  float64
 6   production_companies  21187 non-null  object 
 7   release_date          22551 non-null  object 
 8   budget                22561 non-null  float64
 9   revenue               22561 non-null  float64
 10  runtime               22558 non-null  float64
 11  status                22561 non-null  object 
 12  tagline               16164 non-null  object 
 13  vote_average          22561 non-null  float64
 14  vote_count            22561 non-null  float64
 15  credits            

In [ ]:
movies[['id', 'title', 'poster_path', 'release_date']].to_json("movies_list.json", orient="records")